# Multidimensional data in pandas
We have covered some pandas basics and learned how to plot. Now let's sort out how to deal with more complex data. We will often find ourselves with data in which the unit of observation is complex. Pandas helps us deal with this by allowing for many index variables. So far, we have only used single indexing, but that is about to change. 

Some examples that could use a multiIndex
1. State and country
2. Team and player
3. Industry and firm
4. Country (or person, firm,...) and time

That last one is important, and one that shows up a lot in economics. We call is *panel data*. Panel data is sometimes called longitudinal data. It follows the same firm/person/country over time. 

In [ ]:
import pandas as pd                 # load pandas and shorten it to pd
import datetime as dt               # load datetime and shorten it to dt
import matplotlib.pyplot as plt     # for making figures

In [ ]:
soccer = {'team' : ['Man City', 'Man City', 'Man City', 'Man City', 'Chelsea', 'Chelsea'], 
          'player' : ['Walker', 'Stones', 'Foden', 'Jesus', 'Cahill', 'Pedro'],
          'pos' : ['D', 'D', 'M', 'F', 'D', 'F'],
          'goals' : [1, 0, 0, 1, 0, 3],
          'assists': [0,0,0,0,0,0]
         }

prem = pd.DataFrame(soccer)
prem

### Multiple indexing
The key to working with more complex datasets is getting the index right. So far, we have considered a single index, but pandas allows for multiple indexes that nest each other. 

**Key concept:** Hierarchical indexing takes multiple *levels* of indexes. 

Let's set up the DataFrame to take team and position as the indexes. 

In [ ]:
prem.set_index(['team', 'pos'], inplace=True)
prem

Wow. 

Notice that the `set_index()` method is the same one we used early with single indexes. In this case, we passed it a list of variables to make the indexes
```python
prem.set_index(['team', 'pos'], inplace=True)
```

In the output, the highest level of the index is team (we passed it 'team' first in the list) and the second level is position. The output does not repeat the team name for each observation. The 'missing' team name just mean that the team is the same as above. \[A very Tufte-esque removal of unnecessary ink.\] 

Let's take a look under the hood. What's our index? A new kind of object: the MultiIndex

In [ ]:
print(prem.index)

### Subsetting with multiple indexes
With a multi index, we need two arguments to reference observations

In [ ]:
# All the defenders on Man City
prem.loc[('Man City', 'D'),:] 

It's always a good idea to pay attention to warnings, particularly 'PerformanceWarning'. Pandas is telling us that we are asking for something in the second index, but the second index is not ordered. Let's fix that with `sort_index()`.

**Important** Sort your mulitIndex. 

In [ ]:
prem = prem.sort_index(axis=0)   # tell pandas which axis to sort. Could sort the columns, too...
                                 # returns a DataFrame unless we use inplace=True
prem

In [ ]:
# Now let's ask for all the defenders on Man City
prem.loc[('Man City', 'D'), :]

No warnings. 

### Partial indexing
With the indexes set, we can easily subset the data using only one of the indexes. In pandas, this is called *partial indexing* because we are only using part of the index to subset identify the data we want. 

We can use `loc[]` like we do with a single index if we want to index on the top level index.

In [ ]:
print(prem.loc['Chelsea'])               # All the 'Chelsea' observations
print('\n')
print(prem.loc['Man City'])              # All the 'Man City' observations

#### The xs() method
We can also use the `xs()` method. Here we specify which level we are looking into. Note that I can reference the levels either by an integer or by its name.

In [ ]:
print(prem.xs('Chelsea', level = 0) )              # All the 'Chelsea' observations
print('\n')
print(prem.xs('Man City', level = 'team'))              # All the 'Man City' observations

We can partially index on the 'inner index' as well. Suppose we want all the defenders, regardless of team.

In [ ]:
prem.xs('D', level=1)

As before, we can get rid of the index and replace it with a generic list of integers...

In [ ]:
prem.reset_index(inplace=True)    # this moves the indexes back to columns
prem

...and reset it with three levels of indexes!

In [ ]:
prem.set_index(['team', 'player', 'pos'], inplace=True)
prem

#### A multiIndex in columns
There is nothing that says you can't have multiple indexes in the axis=1 dimension. Here is quick way to see this: transpose the DataFrame.

In [ ]:
prem = prem.transpose()           #this swaps the rows for columns
print(prem)

Now the rows are named 'goals' and 'assists' and the columns are (team, player, pos). I'm not sure this is a very useful way to look at this particular dataset, but multiIndex columns can come in handy. Let's change it back.

In [ ]:
prem = prem.transpose()
print(prem)

### Summary statistics by level
MultiIndexes provide a quick way to summarize data. We will see many different ways to do this --- getting statistics by group --- and not all will involve a multiIndex. 

In [ ]:
# When subsetting by the upppermost level, I can use xs or loc
print('Chelsea avg. goals', prem.xs('Chelsea', level='team')['goals'].mean())   # average goals for Chelsea players
print('Chelsea avg. goals', prem.loc['Chelsea','goals'].mean())   # average goals for Chelsea players

# When subsetting on the inner levels, I use xs 
print('Defender avg. goals {0:.2f}.'.format( prem.xs('D', level='pos')['goals'].mean() ) )          # average goals for defenders

Notice the syntax with xs.
```python
 prem.xs('Chelsea', level='team')['goals']
```

The `prem.xs('Chelsea', level='team')` is returning a DataFrame with all the columns. 

We then use the usual square-bracket syntax to pick off just the column 'goals' and then hit with `mean()`


### Saving multiIndex DataFrames
Saving a multiIndexed DataFrame works like before. Pandas fills in all the repeated labels to the output is ready to go. Run the following code and then open the csv files.

In [ ]:
# Multiple indexes on rows
prem.to_csv('prem.csv')

# Multiple indexes on columns
prem = prem.transpose()
prem.to_csv('prem_transposed.csv')

## Practice

Let's redo question \#3 on the exam using multiIndexes.  

1. Load the march cps data, 'CPS_March_2016.csv'. Remember, missing values are '.'

2. Keep only those with `fulltimely == 1`
3. Keep only those with `5< =hrwage <=200`

4. Rename 'female' to 'sex'
5. In column 'sex' replace 0 with 'male' and 1 with 'female'

6. Set the index to 'sex' and 'educ', in that order.
7. Sort the index. 

8. Report the average wage for males and females. Try it with the `loc[]` method. 

9. Report the average wage for `HS diploma/GED` and for `College degree`, regardless of sex. Use the `xs()` method.  

## Panel data

In [ ]:
# load a data file with the number of walks and snacks my dogs have had 
dogs = pd.read_csv('dogs.csv')         # data on the habits of my dogs
dogs


This data format is called **long** because there are lots of rows and not many columns. Moving between long and **wide** (lots of columns, fewer rows) is a common task in setting up panel data sets. 

Pandas calls long data **stacked** and wide data **unstacked**. We use the `stack()` and `unstack()` methods for moving between long and wide with multiIndexed data. Stack and unstack do not work in place. They always return a copy, so we need to assign it to variables.

In [ ]:
# move everything but the data values to the index
dogs = dogs.set_index(['dog', 'time', 'var'], inplace=False)
dogs

We **unstack the data** to put the variables into columns. 

In [ ]:
dogs_us = dogs.unstack('var')
dogs_us


We can unstack several variables. As usual, we pass a list. 

In [ ]:
dogs_us = dogs.unstack(['dog', 'var'])
dogs_us

We now have an unstacked DataFrame and each column is a time series of one dog's observations. 

Notice that we do not have observations for Thursday's walks, so panda filled in NaNs for us. 

We can **stack the data** to put the variables back on the rows. 

When we unstack the data, pandas defaults to dropping the NaNs. We can override this if we choose. If we do not pass an argument, it stacks the innermost index. 

In [ ]:
dogs_s = dogs_us.stack()       # stack shifts the columns to rows. 
dogs_s

In [ ]:
dogs_s = dogs_us.stack(['dog', 'var'], dropna=False)
dogs_s

In [ ]:
dogs_s=dogs_s.swaplevel('dog', 'time')
dogs_s

In [ ]:
dogs_s=dogs_s.swaplevel('var', 'time')
dogs_s

In [ ]:
dogs_s.sort_index(inplace=True)
dogs_s

## Practice

Let's review multiIndexing with some real world data. The data is messy and will require some cleaning up and 'wrangling.' We will do some of it together, then I'll  hand it off for you all to finish. 

We will work with the IMF's [World Economic Outlook](https://www.imf.org/external/pubs/ft/weo/2017/02/weodata/download.aspx), which contains historical data and the IMF's forecasts for many countries and variables. 

Our **goal** is to study the evolution of debt in Germany, Argentina, and Greece. 

First, download the data file and open it in Excel. The file is here [http://www.imf.org/external/pubs/ft/weo/2016/02/weodata/WEOOct2016all.xls](http://www.imf.org/external/pubs/ft/weo/2016/02/weodata/WEOOct2016all.xls). Clicking on the link should initiate a download.

Wow, there is a lot going on here. Let's get to work.

In [ ]:
url = 'http://www.imf.org/external/pubs/ft/weo/2016/02/weodata/WEOOct2016all.xls'

# The data are in a tab-separated list (even though the file ends in 'xls'. not cool, IMF)
# The encoding parameter tells pandas how to read special characters
# The thousands parameter tells pandas to remove the comma when reading in numbers
weo = pd.read_csv(url, sep='\t', na_values=['n/a', '--'], thousands =',', encoding='windows-1252')
weo.head()


In [ ]:
weo.tail()

The data file has a footer (go back and look in the file) and we read it in as a line of data. We could go back and specify the footer `skipfooter` parameter to read_csv(), but we can also just drop it.

In [ ]:
weo.drop(8404, inplace=True)   # Using the row number is not very robust. What if the IMF add more countries to the workbook?
weo.tail()

In [ ]:
weo.head()

In [ ]:
weo.drop(['WEO Country Code', 'Subject Notes', 'Country/Series-specific Notes', 'Scale', 'Estimates Start After'],axis=1, inplace=True)
weo.head()

In [ ]:
variables = ['GGXWDG_NGDP', 'GGXCNL_NGDP']
countries = ['ARG', 'DEU', 'GRC']

weo = weo[ weo['WEO Subject Code'].isin(variables) & weo['ISO'].isin(countries) ]
weo.head()

### The isin() method
Check out the way that I selected the rows of the dataFrame I wanted. I used the `isin()` method. You pass it a list of labels and it picks them out of the column. This is a lot cleaner than using a bunch of conditional statements.  

I'm using it twice: once to specify the variables I want and once to specify the countries I want. I join the two together with an & operator.  

### Your turn
The data are in pretty decent shape. Take a few minutes and try the following. Feel free to chat with those around if you get stuck. The TA and I are here, too.

1. Rename the 'WEO Subject Code' to 'Variable' and 'Subject Descriptor' to 'Description'.
2. Replace 'GGXCNL_NGDP' with 'Surplus
3. Replace 'GGXWDG_NGDP' with 'Debt'

4. Set the (row) index to be 'ISO', 'Variable', 'Country', 'Description', and 'Units', in that order. 

5. We want the row to be time and everything else to be columns. Swap the columns for rows.
6. Sort the row and column indexes.

7. Plot debt levels for Argentina, Germany and Greece for all the years in the data. Add a legend. Make the figure look nice. Hint: The x-axis is time...

Wow, Greece has had an interesting run. Let's look at Greece more closely. 
8. Create a (2,1) grid of subplots. Plot Greece's surplus in the top axes and debt in the bottom axis. Make it look nice.